## Task 1 Algorithmic Analysis K-Means Clustering with Real World Dataset

First, download a simulated dataset: kmeans_data.zip from Modules->Datasets. Then,
implement the K-means algorithm from scratch. K-means algorithm computes the distance of a
given data point pair. Replace the distance computation function with Euclidean distance, 1-
Cosine similarity, and 1 – the Generalized Jarcard similarity (refer to:
https://www.itl.nist.gov/div898/software/dataplot/refman2/auxillar/jaccard.htm)


Q1: Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the number of categorical values of y (the number of classifications). Compare the SSEs of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better? (10 points)

Q2: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First, label each cluster using the majority vote label of the data points in that cluster. Later, compute the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric is better? (10 points)

Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K- means, Jarcard-K-means. Which method requires more iterations and times to converge? (10 points)

Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to the following three terminating conditions: (10 points)
• when there is no change in centroid position 
• when the SSE value increases in the next iteration 
• when the maximum preset value (e.g., 100) of iteration is complete

Q5: What are your summary observations or takeaways based on your algorithmic analysis? (5 points)


In [1]:
# Import required packages
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import *
from sklearn.cluster import KMeans # to check 

In [2]:
path = 'kmeans_data/'

In [3]:
# Helper functions to calculate distances
def eucl_dist(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

def cos_sim(x1, x2):
    dot_prod = np.dot(x1, x2)
    x1_mag = np.sqrt(np.dot(x1, x1))
    x2_mag = np.sqrt(np.dot(x2, x2))
    mag_prod = x1_mag * x2_mag
    return dot_prod / mag_prod

def cos_sim_dist(x1, x2):
    return 1 - cos_sim(x1, x2)

def gen_jacc_dist(x1, x2):
    intersection = np.minimum(x1, x2).sum()
    union = np.maximum(x1, x2).sum()
    return 1 - (intersection/union)

def init_centroids(X, num_clusters):
    index = random.sample(range(X.shape[0]), num_clusters )
    return X[index, :]

def find_distances(X, centroids, dist_measure):
    dists = np.zeros((X.shape[0], centroids.shape[0]))
    for i, centroid in enumerate(centroids):
        for j, point in enumerate(X):
            dists[j, i] = dist_measure(point, centroid)
    return dists

def assign_cluster(dists):
    return np.argmin(dists, axis=1)


def update_centroids(X, clusters, num_clusters):
    centroids = np.zeros((num_clusters, X.shape[1]))
    for i in range(num_clusters):
        X_i = X[clusters == i] # filters points that belong to cluster 'i' from X
        centroids[i, :] = np.mean(X_i, axis=0)
    return centroids

def find_sse(X, clusters, centroids):
    sse = 0
    for i in range(centroids.shape[0]):
        cl_data = X[clusters == i]
        centroid = centroids[i]
        sse += np.sum((cl_data - centroid)**2)
    return sse

In [4]:
# simple K-means implementation
def kmeans(X, num_clusters, max_iters, dist_measure):
    centroids = init_centroids(X, num_clusters)
    for _ in range(max_iters):
        dists = find_distances(X, centroids, dist_measure)
        clusters = assign_cluster(dists)
        centroids = update_centroids(X, clusters, num_clusters)
    return centroids, clusters

In [5]:
X_df = pd.read_csv(path+"data.csv")
y_df = pd.read_csv(path+"label.csv")

In [6]:
# normalize and convert to numpy array
X_norm = pd.DataFrame(MinMaxScaler().fit_transform(X_df), columns = X_df.columns)

X = X_norm.to_numpy()
y = y_df.to_numpy()

print(X.shape)
print(y.shape)

(9999, 784)
(9999, 1)


In [7]:
K = len(np.unique(y))
max_iters = 100

In [8]:
euc_centroids, euc_clusters = kmeans(X, K, max_iters, eucl_dist)
cos_centroids, cos_clusters = kmeans(X, K, max_iters, cos_sim_dist)
jacc_centroids, jacc_clusters = kmeans(X, K, max_iters, gen_jacc_dist)

#### Q1 - Comparing metrics for sum of squared error

In [9]:
euc_sse = find_sse(X, euc_clusters, euc_centroids)
cos_sse = find_sse(X, cos_clusters, cos_centroids)
jacc_sse = find_sse(X, jacc_clusters, jacc_centroids)

In [10]:
print("Euclidean SSE: {euc_sse}".format(euc_sse=euc_sse))
print("Cosine SSE: {cos_sse}".format(cos_sse=cos_sse))
print("Jaccard SSE: {jacc_sse}".format(jacc_sse=jacc_sse))

Euclidean SSE: 389498.16627122037
Cosine SSE: 390966.4600923645
Jaccard SSE: 390913.889311961


In [11]:
kmeans = KMeans(n_clusters=K, init='k-means++', max_iter=max_iters, random_state=42)
kmeans.fit(X)

sklearn_impl_sse = kmeans.inertia_
print("Custom impl: {e_sse}, sklearn impl: {sk_sse}".format(e_sse=euc_sse, sk_sse=sklearn_impl_sse))

Custom impl: 389498.16627122037, sklearn impl: 389525.15401128225


In [12]:
def labels_majority_vote(X, y, clusters, centroids):
    cluster_labels = np.zeros(centroids.shape[0], dtype=int)
    for i in range(centroids.shape[0]):
        labels_true = y[clusters==i]
        labels_unique, label_counts = np.unique(labels_true, return_counts=True)
        counts_max = np.argmax(label_counts)
        cluster_labels[i] = labels_unique[counts_max]
    return cluster_labels

def compute_accuracy(X, y, clusters, centroids, cluster_labels):
    accurate = 0
    total = len(y)
    for i in range(centroids.shape[0]):
        cl_i = cluster_labels[i]
        y_fl = y[clusters==i] # filters values that belong to cluster 'i' from 'y'
        accurate = accurate + np.sum(y_fl == cl_i)
    return accurate/total

In [13]:
# compute labels
euc_cl_labels = labels_majority_vote(X, y, euc_clusters, euc_centroids)
cos_cl_labels = labels_majority_vote(X, y, cos_clusters, cos_centroids)
jacc_cl_labels = labels_majority_vote(X, y, jacc_clusters, jacc_centroids)

In [14]:
# compute accuracy
euc_accuracy = compute_accuracy(X, y, euc_clusters, euc_centroids, euc_cl_labels)
cos_accuracy = compute_accuracy(X, y, cos_clusters, cos_centroids, cos_cl_labels)
jacc_accuracy = compute_accuracy(X, y, jacc_clusters, jacc_centroids, jacc_cl_labels)

In [15]:
print("Euclidean Accuracy: {euc_accuracy}".format(euc_accuracy=euc_accuracy))
print("Cosine Accuracy: {cos_accuracy}".format(cos_accuracy=cos_accuracy))
print("Jaccard Accuracy: {jacc_accuracy}".format(jacc_accuracy=jacc_accuracy))

Euclidean Accuracy: 0.603060306030603
Cosine Accuracy: 0.6134613461346134
Jaccard Accuracy: 0.6051605160516051


In [16]:
# change of stop criteria
def kmeans_mod_1(X, num_clusters, max_iters, dist_measure, tol=1e-4):
    centroids = init_centroids(X, num_clusters)
    prev_sse = float('inf') # keep track 
    for itr in range(max_iters):
        dists = find_distances(X, centroids, dist_measure)
        clusters = assign_cluster(dists)
        centroids_new = update_centroids(X, clusters, num_clusters)
        change = np.linalg.norm(centroids_new - centroids)
        if change < tol:
            print("centroid change below tolerance")
            break
        centroids = centroids_new
        sse_new = find_sse(X, clusters, centroids)
        if sse_new > prev_sse:
            print("new sse larger than previous sse")
            break
        prev_sse = sse_new        
    return prev_sse, itr+1

In [17]:
max_iters = 500
euc_sse, euc_itr = kmeans_mod_1(X, K, max_iters, eucl_dist)
cos_sse, cos_itr = kmeans_mod_1(X, K, max_iters, cos_sim_dist)
jacc_sse, jacc_itr = kmeans_mod_1(X, K, max_iters, gen_jacc_dist)

centroid change below tolerance
new sse larger than previous sse
new sse larger than previous sse


In [18]:
print("Euclidean iterations:", euc_itr)
print("Cosine iterations:", cos_itr)
print("Jaccard iterations:", jacc_itr)

Euclidean iterations: 76
Cosine iterations: 24
Jaccard iterations: 20


In [19]:
def kmeans_no_change_centroid(X, num_clusters, max_iters, dist_measure, tol=1e-4):
    centroids = init_centroids(X, num_clusters)
    prev_sse = float('inf') # keep track 
    for itr in range(max_iters):
        dists = find_distances(X, centroids, dist_measure)
        clusters = assign_cluster(dists)
        centroids_new = update_centroids(X, clusters, num_clusters)
        change = np.linalg.norm(centroids_new - centroids)
        if change < tol:
            print("centroid change below tolerance")
            break
        centroids = centroids_new
        sse_new = find_sse(X, clusters, centroids)
#         if sse_new > prev_sse:
#             print("new sse larger than previous sse")
#             break
        prev_sse = sse_new        
    return prev_sse, itr+1

def kmeans_sse_increase(X, num_clusters, max_iters, dist_measure, tol=1e-100):
    centroids = init_centroids(X, num_clusters)
    prev_sse = float('inf') # keep track 
    for itr in range(max_iters):
        dists = find_distances(X, centroids, dist_measure)
        clusters = assign_cluster(dists)
        centroids_new = update_centroids(X, clusters, num_clusters)
#         change = np.linalg.norm(centroids_new - centroids)
#         if change < tol:
#             print("centroid change below tolerance")
#             break
        centroids = centroids_new
        sse_new = find_sse(X, clusters, centroids)
        if sse_new > prev_sse:
            print("new sse larger than previous sse")
            break
        prev_sse = sse_new        
    return prev_sse, itr+1

def kmeans_max_itrs_complete(X, num_clusters, max_iters, dist_measure, tol=1e-100):
    centroids = init_centroids(X, num_clusters)
    prev_sse = float('inf') # keep track 
    for itr in range(max_iters):
        dists = find_distances(X, centroids, dist_measure)
        clusters = assign_cluster(dists)
        centroids_new = update_centroids(X, clusters, num_clusters)
        change = np.linalg.norm(centroids_new - centroids)
        if change < tol:
            print("centroid change below tolerance")
            break
        centroids = centroids_new
        sse_new = find_sse(X, clusters, centroids)
#         if sse_new > prev_sse:
#             print("new sse larger than previous sse")
#             break
        prev_sse = sse_new        
    return prev_sse, itr+1

In [20]:
# Q4 terminating condition: no change centroid
max_iters = 200
euc_sse, euc_itr = kmeans_no_change_centroid(X, K, max_iters, eucl_dist)
cos_sse, cos_itr = kmeans_no_change_centroid(X, K, max_iters, cos_sim_dist)
jacc_sse, jacc_itr = kmeans_no_change_centroid(X, K, max_iters, gen_jacc_dist)

print("Euclidean iterations:", euc_itr)
print("Cosine iterations:", cos_itr)
print("Jaccard iterations:", jacc_itr)
print("Euclidean SSE: {euc_sse}".format(euc_sse=euc_sse))
print("Cosine SSE: {cos_sse}".format(cos_sse=cos_sse))
print("Jaccard SSE: {jacc_sse}".format(jacc_sse=jacc_sse))



centroid change below tolerance
centroid change below tolerance
centroid change below tolerance
Euclidean iterations: 71
Cosine iterations: 135
Jaccard iterations: 32
Euclidean SSE: 389408.06798454165
Cosine SSE: 390956.1552859607
Jaccard SSE: 390949.00344651745


In [21]:
# Q4 terminating condition: sse value increases
max_iters = 200
euc_sse, euc_itr = kmeans_sse_increase(X, K, max_iters, eucl_dist)
cos_sse, cos_itr = kmeans_sse_increase(X, K, max_iters, cos_sim_dist)
jacc_sse, jacc_itr = kmeans_sse_increase(X, K, max_iters, gen_jacc_dist)

print("Euclidean iterations:", euc_itr)
print("Cosine iterations:", cos_itr)
print("Jaccard iterations:", jacc_itr)
print("Euclidean SSE: {euc_sse}".format(euc_sse=euc_sse))
print("Cosine SSE: {cos_sse}".format(cos_sse=cos_sse))
print("Jaccard SSE: {jacc_sse}".format(jacc_sse=jacc_sse))

new sse larger than previous sse
new sse larger than previous sse
Euclidean iterations: 200
Cosine iterations: 28
Jaccard iterations: 27
Euclidean SSE: 389410.68557001994
Cosine SSE: 392558.53735727724
Jaccard SSE: 391066.484809759


In [22]:
# Q4 terminating condition: max preset value iter is complete
max_iters = 50
euc_sse, euc_itr = kmeans_max_itrs_complete(X, K, max_iters, eucl_dist)
cos_sse, cos_itr = kmeans_max_itrs_complete(X, K, max_iters, cos_sim_dist)
jacc_sse, jacc_itr = kmeans_max_itrs_complete(X, K, max_iters, gen_jacc_dist)

print("Euclidean iterations:", euc_itr)
print("Cosine iterations:", cos_itr)
print("Jaccard iterations:", jacc_itr)
print("Euclidean SSE: {euc_sse}".format(euc_sse=euc_sse))
print("Cosine SSE: {cos_sse}".format(cos_sse=cos_sse))
print("Jaccard SSE: {jacc_sse}".format(jacc_sse=jacc_sse))

Euclidean iterations: 50
Cosine iterations: 50
Jaccard iterations: 50
Euclidean SSE: 390196.9929245067
Cosine SSE: 391622.01637264126
Jaccard SSE: 390923.6038410168
